In [3]:
import openai
import cohere
import os
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.http.models import models
from dotenv import load_dotenv

load_dotenv()

collection_name = "hackathon_collection"

# Title: The Will to Believe and Other Essays in Popular Philosophy
# Author: William James

def cohere_embedding(question):
    cohere_client = cohere.Client(os.environ.get('COHERE_API_KEY'))
    model = 'multilingual-22-12'
    response = cohere_client.embed(
        texts=[question],
        model=model,
    )
    embedding = [float(e) for e in response.embeddings[0]]
    return embedding

def openai_response(question, answers_list):
    prompt = f"""
        Given the texts below, answer the following question:
        Question: {question}

        Texts:
        """
    for answer in answers_list:
        prompt += '{}\n'.format(answer)
    
    messages = [{"role": "user", "content": prompt}]
    
    openai_model = 'gpt-3.5-turbo'
    response = openai.ChatCompletion.create(
        model=openai_model,
        messages=messages,
        temperature=0.1,
        max_tokens=1000,
        # frequency_penalty=0.0,
        # presence_penalty=0.0,
        # stop=["\n"]
    )

    # open_ai_response.append(response.choices[0].message.content)
    print(prompt)
    return response.choices[0].message.content
    

def qdrant_search_by_filter(key, value, question):
    # perform author payload filter + information vector search
    embedding = cohere_embedding(question)
    db_client = QdrantClient(
        api_key=os.environ.get('QDRANT_API_KEY'),
        host=os.environ.get('QDRANT_HOST')
    )
    response = db_client.search(
        collection_name=collection_name,
            query_filter=models.Filter(
                must=[
                    models.FieldCondition(
                        key=key,
                        match=models.MatchValue(
                            value=value
                        ) 
                    )
                ]
            ),
        query_vector=embedding,
        limit=5
    )

    return response

def search_author(input):
    author_info, question_info = input.split('AUTHOR:', 1)[1].split('INFORMATION:', 1)
    author = author_info.strip().lower()
    question = question_info.strip().lower()
    qdrant_answer = qdrant_search_by_filter(key='author', value=author, question=question)
    answers_list = []
    for count, answer in enumerate(qdrant_answer):
        if count == 0:
            author_name = answer.payload.get('author', 'unknown')
            answers_list.append(f"author name is {author_name}")    
        answers_list.append(f"{answer.payload.get('text')}")
    
    return openai_response(question, answers_list)


def search_title(input):
    # perform title payload filter + information vector search
    title_info, question_info = input.split('TITLE:', 1)[1].split('INFORMATION:', 1)
    title = title_info.strip().lower()
    question = question_info.strip().lower()
    qdrant_answer = qdrant_search_by_filter(key='title', value=title, question=question)
    answers_list = []
    for count, answer in enumerate(qdrant_answer):
        if count == 0:
            title = answer.payload.get('title', 'unknown')
            answers_list.append(f"title is {title}")
        answers_list.append(f"{answer.payload.get('text')}")
    
    return openai_response(question, answers_list)
        
    

def qdrant_search(question):
    embedding = cohere_embedding(question)

    db_client = QdrantClient(
        api_key=os.environ.get('QDRANT_API_KEY'),
        host=os.environ.get('QDRANT_HOST')
    )
    qdrant_answer = db_client.search(
        collection_name="hackaton_collection",
        query_vector=embedding,
        limit=5,
    )

    answers_list = []
    for answer in qdrant_answer:
        answers_list.append('{}\n'.format(answer.payload.get('text')))

    return openai_response(question, answers_list)


tools = [
    Tool(
        name="Search",
        func=lambda question: qdrant_search(question),
        description="""Use this more often than other tools, when you need 
        to answer questions and do not have the author's name or the document's title"""
        # description="use when searching for information filtering by a specific author.",
        # description="use when you want to discover who is the author, asking a question with informations you have",
    ),
    Tool(
        name="Search when knowing specific author name",
        func=lambda input: search_author(input),
        description="""Use this only when you know one specific author's name and you are searching for information about this author.
        Do not use this if you do not know the author's name.
        Create an input with the name of the author and the information you are searching for them.
        Input template: 'AUTHOR: name of the author INFORMATION: other information you are searching'"""
        # description="use when you know the author's name and want to filter results based on their name and other informations that you have. create input like 'author: information:'"
        # description="use when searching for information filtering by a specific author.",
        # description="use when you want to discover who is the author, asking a question with informations you have",
    ),
    Tool(
        name="Search when knowing specific document title",
        func=lambda input: search_title(input),
        description="""Use this only when you are searching for information about one specific document title 
        and you know this document's title.
        Do not use this if you do not know the document's title.
        Create an input with the title of the document and the information you are searching for them.
        Input template: 'TITLE: title of the document INFORMATION: other information you are searching'"""
        # description="use when searching for information filtering by a specific title.",
        # description="use when you want to discover which is the title, asking a quesiton with informations you have",
    )
]

agent = initialize_agent(
    tools=tools, 
    llm=OpenAI(temperature=0), 
    agent="zero-shot-react-description", 
    verbose=True,
    # return_intermediate_steps=True
)



In [4]:
# question = 'who wrote about his posthumous memories?'
# question = 'quem é o autor que escreve sobre suas memórias póstumas?'
question = 'em qual trecho o machado de assis comenta sobre filhos?'
# question = 'qual é o titulo do livro em o autor machado de assis comenta sobre filhos?'
# question = 'Com efeito, um dia de manhã, estando a passear na chácara, pendurou-se-me uma idéia no trapézio que eu tinha no cérebro.']

agent.run(input=question)
# response = agent({"input": question})
# response["intermediate_steps"]




> Entering new AgentExecutor chain...
 I need to find a document by Machado de Assis that mentions something about children.
Action: Search when knowing specific author name
Action Input: AUTHOR: Machado de Assis INFORMATION: mentions something about children
        Given the texts below, answer the following question:
        Question: mentions something about children

        Texts:
        author name is machado de assis
— É muito esperto o seu menino ! exclamavam os ouvintes.      — Muito esperto, concordava meu pa i; e os olhos babavam-se-lhe de  orgulho, e ele espalmava a mão sobre a minha cabeça, fitava-me  longo tempo, namorado, cheio de si.      Item, comecei a andar, não sei bem quando, mas antes do tempo.  Talvez por apressar a natureza, obrigavam-me cedo a agarrar às  cadeiras, pegavam-me da fralda, davam-me carrinhos de pau. — Só  só, nhonhô, só só, dizia-me a muca ma. E eu, atraído pelo chocalho  de lata, que minha mãe agitava diante de mim, lá ia para a frente,  cai 

'Machado de Assis menciona crianças no primeiro texto, onde descreve sua própria infância e como aprendeu a andar cedo.'

In [103]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: Use this more often than other tools, when you need 
        to answer questions and do not have the author's name or the document's title
Search when knowing specific author name: Use this only when you know one specific author's name and you are searching for information about this author.
        Do not use this if you do not know the author's name.
        Create an input with the name of the author and the information you are searching for them.
        Input template: 'AUTHOR: name of the author INFORMATION: other information you are searching'
Search when knowing specific document title: Use this only when you are searching for information about one specific document title 
        and you know this document's title.
        Do not use this if you do not know the document's title.
        Create an input with the title of the document and the information you are searching for them.
 

In [104]:
agent.agent.plan

<bound method Agent.plan of ZeroShotAgent(llm_chain=LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f4d49f5d3f0>, verbose=False, prompt=PromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={}, template="Answer the following questions as best you can. You have access to the following tools:\n\nSearch: Use this more often than other tools, when you need \n        to answer questions and do not have the author's name or the document's title\nSearch when knowing specific author name: Use this only when you know one specific author's name and you are searching for information about this author.\n        Do not use this if you do not know the author's name.\n        Create an input with the name of the author and the information you are searching for them.\n        Input template: 'AUTHOR: name of the author INFORMATION: other information you are searching'\nSearch when knowing specific docum

In [ ]:
from langchain.agents.agent import AgentExecutor
from langchain.agents.loading import AGENT_TO_CLASS, load_agent
from langchain.agents.mrkl.base import ZeroShotAgent
from langchain.agents.mrkl.prompt import FORMAT_INSTRUCTIONS, PREFIX, SUFFIX
from langchain.schema import AgentAction, AgentFinish

In [28]:
search_author('AUTHOR: machado de assis INFORMATIONS: em qual trecho o machado de assis comenta sobre filhos?')


        Given the texts below, answer the following question:
        Question: em qual trecho o machado de assis comenta sobre filhos?

        Texts:
        author name is machado de assis
— É muito esperto o seu menino ! exclamavam os ouvintes.      — Muito esperto, concordava meu pa i; e os olhos babavam-se-lhe de  orgulho, e ele espalmava a mão sobre a minha cabeça, fitava-me  longo tempo, namorado, cheio de si.      Item, comecei a andar, não sei bem quando, mas antes do tempo.  Talvez por apressar a natureza, obrigavam-me cedo a agarrar às  cadeiras, pegavam-me da fralda, davam-me carrinhos de pau. — Só  só, nhonhô, só só, dizia-me a muca ma. E eu, atraído pelo chocalho  de lata, que minha mãe agitava diante de mim, lá ia para a frente,  cai aqui, cai acolá; e andava, provavelmente mal, mas andava, e  fiquei andando.           CAPÍTULO XI / O MENINO É PAI DO HOMEM      Cresci; e nisso é que a família não interveio; cresci naturalmente,  como crescem as magnólias e os gatos. Ta

'Não há um trecho específico em que Machado de Assis comenta sobre filhos.'